In [32]:
import os
import math
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import math

In [33]:
from sklearn.discriminant_analysis import StandardScaler


class CustomDataset(Dataset):
    def __init__(self, csv_path="data/dataset_train_2024.csv"):
        # Load data from CSV
        data = pd.read_csv(csv_path)
        
        # Extract features
        self.sequences_1 = data.iloc[:, 1:129].values * 100  # Columns 1-128 (1-based indexing)
        self.sequences_2 = data.iloc[:, 129:257].values * 100  # Columns 129-256
        self.extra_feature = data.iloc[:, 257].values.reshape(-1, 1)  # Column 257

        # Combine features
        all_features = np.hstack([self.sequences_1, self.sequences_2, self.extra_feature])
        
        # Normalize features
        self.scaler = StandardScaler()
        self.normalized_features = self.scaler.fit_transform(all_features)
        self.features = torch.tensor(self.normalized_features, dtype=torch.float32)


        # Encode labels
        self.label_encoder = LabelEncoder()
        self.labels = torch.tensor(self.label_encoder.fit_transform(data.iloc[:, -1]), dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
    def inverseTransform(self, array):
        return self.label_encoder.inverse_transform(array)

In [50]:
class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional=False, dropout=0.0):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.rnn = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout
        )
        # Adjust the output dimension if bidirectional is used
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x):
        # RNN forward pass
        # Input shape: [batch_size, seq_len, input_dim]
        out, _ = self.rnn(x)  # out: [batch_size, seq_len, hidden_dim]
        # Use the last hidden state for classification
        out = out[:, -1, :]  # [batch_size, hidden_dim]
        out = self.fc(out)  # [batch_size, num_classes]
        return out


In [35]:
def reformat_tensor(tensor):
    """
    Reformats a tensor from shape [batch_size, 257] to [batch_size, 128, 3] for RNN input.
    """
    batch_size = tensor.shape[0]
    # Extract sequences
    seq1 = tensor[:, :128]  # First 128 features
    seq2 = tensor[:, 128:256]  # Next 128 features
    # Extract noise and expand it to match seq_len
    noise = tensor[:, -1].unsqueeze(1).expand(batch_size, 128)
    # Stack along the last dimension to form [batch_size, 128, 3]
    return torch.stack([seq1, seq2, noise], dim=2)

# Hyperparameters

hidden_dim = 64

num_layers = 2

bidirectional = True  # Set to False for unidirectional RNN

num_classes = 5

learning_rate = 0.001

epochs = 50

batch_size = 32

In [63]:
# Hyperparameters
hidden_dim = 32
num_layers = 3
bidirectional = True  # Set to False for unidirectional RNN
num_classes = 5
learning_rate = 0.001
epochs = 50
batch_size = 16

# Prepare Data
dataset = CustomDataset()
train_size = int(len(dataset))  # 80% for training
test_size = int(len(dataset) * 0.2)  # 20% for testing
train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Model Initialization
model = RNNClassifier(
    input_dim=3,  # This matches the last dimension of [batch_size, 128, 3]
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional
).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
print("Training RNN model...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        features = reformat_tensor(features).to(device)  # Reshape for RNN
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_loader:
        features = reformat_tensor(features).to(device)
        labels = labels.to(device)

        outputs = model(features)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


Using device: cuda
Training RNN model...
Epoch 1/50, Loss: 1.4799
Epoch 2/50, Loss: 0.6238
Epoch 3/50, Loss: 0.3860
Epoch 4/50, Loss: 0.3252
Epoch 5/50, Loss: 0.3071
Epoch 6/50, Loss: 0.2902
Epoch 7/50, Loss: 0.2738
Epoch 8/50, Loss: 0.2418
Epoch 9/50, Loss: 0.2013
Epoch 10/50, Loss: 0.1747
Epoch 11/50, Loss: 0.1427
Epoch 12/50, Loss: 0.1190
Epoch 13/50, Loss: 0.0984
Epoch 14/50, Loss: 0.0838
Epoch 15/50, Loss: 0.0677
Epoch 16/50, Loss: 0.0627
Epoch 17/50, Loss: 0.0534
Epoch 18/50, Loss: 0.0443
Epoch 19/50, Loss: 0.0586
Epoch 20/50, Loss: 0.0356
Epoch 21/50, Loss: 0.0364
Epoch 22/50, Loss: 0.0330
Epoch 23/50, Loss: 0.0400
Epoch 24/50, Loss: 0.0247
Epoch 25/50, Loss: 0.0450
Epoch 26/50, Loss: 0.0193
Epoch 27/50, Loss: 0.0213
Epoch 28/50, Loss: 0.0141
Epoch 29/50, Loss: 0.0224
Epoch 30/50, Loss: 0.0175
Epoch 31/50, Loss: 0.0147
Epoch 32/50, Loss: 0.0270
Epoch 33/50, Loss: 0.0196
Epoch 34/50, Loss: 0.0227
Epoch 35/50, Loss: 0.0126
Epoch 36/50, Loss: 0.0094
Epoch 37/50, Loss: 0.0136
Epoch 

In [64]:
# Testing Loop
print("Testing the model...")
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_loader:
        # Move data to the appropriate device
        features, labels = features.to(device), labels.to(device)
        
        # Reformat the input tensor to match the expected RNN input shape
        features = reformat_tensor(features)
        
        # Get model predictions
        outputs = model(features)
        _, preds = torch.max(outputs, dim=1)
        
        # Collect predictions and labels for evaluation
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Evaluate performance
accuracy = accuracy_score(all_labels, all_preds)
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_preds, average='weighted')  # or 'macro', 'micro', depending on your use case
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Testing the model...
F1 Score: 0.9812
Accuracy: 0.9812


# Prepare for Kaggle


In [65]:
#Using the model for prediction with the evaluation dataset

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Define the dataset class
class UnlabeledDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe

        self.scaler = StandardScaler()
        normalized_values = self.scaler.fit_transform(self.data.values)
        self.normalized_data = pd.DataFrame(
            normalized_values, columns=self.data.columns, index=self.data.index
        )
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.normalized_data.iloc[idx].values.astype('float32')  # Retrieve row as NumPy array
        if self.transform:
            inputs = self.transform(inputs)
        return inputs

# Load the unlabeled dataset
csv_path = "data/dataset_test_no_label_2024.csv"  # Path to the dataset CSV file
unlabeled_df = pd.read_csv(csv_path)  # Update the filename
unlabeled_df = unlabeled_df.drop(unlabeled_df.columns[0], axis=1)
unlabeled_dataset = UnlabeledDataset(unlabeled_df)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Store predictions and indices
predictions = []
indices = []

# Generate predictions
with torch.no_grad():
    for idx, inputs in enumerate(unlabeled_dataloader):
        inputs = inputs.to(device)  # Send inputs to the same device as the model
        
        # Reformat inputs to match RNN input shape
        inputs = reformat_tensor(inputs)
        
        # Forward pass
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)  # Get predicted class
        
        # Save predictions and indices
        start_idx = idx * unlabeled_dataloader.batch_size
        batch_indices = list(range(start_idx, start_idx + len(inputs)))  # Adjusting the index properly
        indices.extend(batch_indices)
        predictions.extend(preds.cpu().numpy())

# Create a DataFrame with indices and predictions
output_df = pd.DataFrame({"ID": indices, "MODULATION": dataset.inverseTransform(predictions)})

# Save to a CSV file
output_df.to_csv("predictions_RNN.csv", index=False)

print("Predictions saved to 'predictions_RNN.csv'")

Predictions saved to 'predictions_RNN.csv'


# Save the model

In [66]:
# Save the model's state dictionary
model_path = "./RNN/rnn_classifier_gridSearch.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to ./RNN/rnn_classifier_gridSearch.pth


To load the saved model we use 

In [ ]:
# Define parameters
input_dim = 3  # Match your input dimensions
hidden_dim = 128  # Hidden dimension of your RNN
num_layers = 2  # Number of RNN layers
num_classes = 5  # Number of output classes
bidirectional = False  # Match your model setting

# Recreate the model
loaded_model = RNNClassifier(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional
)

# Load the state dictionary
loaded_model.load_state_dict(torch.load("rnn_classifier.pth"))
loaded_model.to(device)  # Send the model to the appropriate device
loaded_model.eval()  # Set the model to evaluation mode

print("Model loaded successfully!")

# GridSearch


In [57]:
def train_model(model, train_loader, val_loader, epochs, learning_rate, batch_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            features = reformat_tensor(features)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            features = reformat_tensor(features)
            outputs = model(features)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    score = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, score

In [61]:
from itertools import product

hidden_dims = [32, 64, 128, 256, 512, 1024, 2048]
num_layers = [1, 2, 3, 4]
learning_rates = [0.001, 0.01]
batch_sizes = [16, 32, 64, 128, 256, 512, 1024]
dropouts = [0.0, 0.2, 0.5]
bidirectionals = [False, True]
    
best_accuracy = 0
best_f1_score = 0
best_params = None

# Prepare Data
dataset = CustomDataset()
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing
train_data, test_data = random_split(dataset, [train_size, test_size])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Generate all combinations of hyperparameters
hyperparameter_combinations = product(hidden_dims, num_layers, learning_rates, batch_sizes, dropouts, bidirectionals)

for hidden_dim, num_layer, learning_rate, batch_size, dropout, bidirectional in hyperparameter_combinations:
        print(f"Training with hidden_dim={hidden_dim}, num_layers={num_layer}, lr={learning_rate}, batch_size={batch_size}, dropout={dropout}, bidirectional={bidirectional}")
        
        # Create DataLoader for current batch_size
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

        # Initialize and train the model
        model = RNNClassifier(input_dim=3, hidden_dim=hidden_dim, num_layers=num_layer, num_classes=num_classes, 
                              bidirectional=bidirectional, dropout=dropout).to(device)
        
        accuracy, score = train_model(model, train_loader, val_loader, epochs, learning_rate, batch_size)
        
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation Accuracy: {score:.4f}")
        
        # Update best accuracy and parameters
        if score > best_f1_score:
            best_accuracy = accuracy
            best_f1_score = score
            best_params = {
                  'hidden_dim': hidden_dim,
                  'num_layers': num_layer,
                  'learning_rate': learning_rate,
                  'batch_size': batch_size,
                  'dropout': dropout,
                  'bidirectional': bidirectional
                  }

print("Best Parameters found:", best_params)
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation F1-Score: {best_f1_score:.4f}")

Using device: cuda
Training with hidden_dim=32, num_layers=1, lr=0.001, batch_size=16, dropout=0.0, bidirectional=False
Validation Accuracy: 0.8796
Validation Accuracy: 0.8784
Training with hidden_dim=32, num_layers=1, lr=0.001, batch_size=16, dropout=0.0, bidirectional=True
Validation Accuracy: 0.8292
Validation Accuracy: 0.8280
Training with hidden_dim=32, num_layers=1, lr=0.001, batch_size=16, dropout=0.2, bidirectional=False


c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Validation Accuracy: 0.9117
Validation Accuracy: 0.9110
Training with hidden_dim=32, num_layers=1, lr=0.001, batch_size=16, dropout=0.2, bidirectional=True


c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Validation Accuracy: 0.8608
Validation Accuracy: 0.8599
Training with hidden_dim=32, num_layers=1, lr=0.001, batch_size=16, dropout=0.5, bidirectional=False


...

c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Validation Accuracy: 0.8646
Validation Accuracy: 0.8633
Training with hidden_dim=32, num_layers=1, lr=0.01, batch_size=1024, dropout=0.5, bidirectional=True


c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Validation Accuracy: 0.8438
Validation Accuracy: 0.8412
Training with hidden_dim=32, num_layers=2, lr=0.001, batch_size=16, dropout=0.0, bidirectional=False
Validation Accuracy: 0.9729
Validation Accuracy: 0.9729
Training with hidden_dim=32, num_layers=2, lr=0.001, batch_size=16, dropout=0.0, bidirectional=True
Validation Accuracy: 0.9604
Validation Accuracy: 0.9606
Training with hidden_dim=32, num_layers=2, lr=0.001, batch_size=16, dropout=0.2, bidirectional=False
Validation Accuracy: 0.9688
Validation Accuracy: 0.9688
Training with hidden_dim=32, num_layers=2, lr=0.001, batch_size=16, dropout=0.2, bidirectional=True
Validation Accuracy: 0.9779
Validation Accuracy: 0.9779
Training with hidden_dim=32, num_layers=2, lr=0.001, batch_size=16, dropout=0.5, bidirectional=False
Validation Accuracy: 0.9671
Validation Accuracy: 0.9670
Training with hidden_dim=32, num_layers=2, lr=0.001, batch_size=16, dropout=0.5, bidirectional=True
Validation Accuracy: 0.9758
Validation Accuracy: 0.9759
Train

KeyboardInterrupt: 

In [62]:
print("Best Parameters found:", best_params)
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation F1-Score: {best_f1_score:.4f}")

Best Parameters found: {'hidden_dim': 32, 'num_layers': 3, 'learning_rate': 0.001, 'batch_size': 16, 'dropout': 0.0, 'bidirectional': True}
Best Validation Accuracy: 0.9900
Best Validation F1-Score: 0.9900


Best Parameters found: {'hidden_dim': 32, 'num_layers': 3, 'learning_rate': 0.001, 'batch_size': 16, 
'dropout': 0.0, 'bidirectional': True}

Best Validation Accuracy: 0.9900

Best Validation F1-Score: 0.9900